# Analysis of top preventable causes of death by MSOA in England

SDC Coursework 

In this analysis proportions of deaths by preventable causes, as defined by the [Office for National Statistics (ONS)](https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/bulletins/avoidablemortalityinenglandandwales/2018#measuring-the-data). All data used in this analysis can be found on the relevant [Github Repo](https://github.com/signesw/Seeya_later). The causes of death were sourved from the [Nomis Portal](https://www.nomisweb.co.uk/query/construct/components/simpleapicomponent.aspx?menuopt=1613&subcomp=).

Yasmine Hujair and Emmanuel Farinre (Data team).

Any questions speak to [Yasmine Hujair](yasmine.hujair@gmail.com)

## Read in data 

In [1]:
#Importing packages

import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm                                                                                                                                                     
import matplotlib as mpl
import re
import os

import sklearn
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN, OPTICS
from esda.adbscan import ADBSCAN

import random
random.seed(42)    # For reproducibility
np.random.seed(42) # For reproducibility

# Make numeric display a bit neater
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

In [2]:
# Set download URL
url = 'https://raw.githubusercontent.com/signesw/Seeya_later/main/data/Causes/Deaths-by-preventable-causes-counts.csv'
#read in csv
counts = pd.read_csv(url, low_memory=False)

#set index
counts.set_index(['MSOA Code'], inplace = True)
counts = counts.drop(['MSOA Name'], axis = 1)
#drop first row
counts = counts[1:]
counts.head()

,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,...,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82
MSOA Code,,,,,,,,,,,,,,,,,,,,,
E02001347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E02001348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E02001349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E02001350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E02001351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
url = 'https://raw.githubusercontent.com/signesw/Seeya_later/main/data/Causes/Deaths-by-preventable-causes.csv'
causes = pd.read_csv(url, low_memory=False)
print(f"Data frame is {causes.shape[0]:,} x {causes.shape[1]}")
causes.head()

Data frame is 876 x 6


,Condition Group,Cause,ICD-10 codes,Age,Treatable,Preventable
0,Infectious diseases,Intestinal diseases,A00,0-74,0.00,1.00
1,Infectious diseases,Intestinal diseases,A01,0-74,0.00,1.00
2,Infectious diseases,Intestinal diseases,A02,0-74,0.00,1.00
3,Infectious diseases,Intestinal diseases,A03,0-74,0.00,1.00
4,Infectious diseases,Intestinal diseases,A04,0-74,0.00,1.00


# Data Wrangling

In [4]:
#convert columns to floats
for col in counts:
    counts[col] = counts[col].astype('float64')

In [5]:
#transpose counts
counts_t = counts.T
msoas = counts_t.columns.values.tolist()
#add total columns
counts_t['Total'] = counts_t.sum(axis=1)
counts_t.head()

MSOA Code,E02001347,E02001348,E02001349,E02001350,E02001351,E02001352,E02001353,E02001354,E02001355,E02001356,...,E02005123,E02005124,E02005125,E02005126,E02005127,E02005128,E02005129,E02005130,E02005131,Total
A00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00


In [6]:
#sort by top values
counts_t.sort_values(by=['Total'],ascending=False, inplace=True)
counts_t.head(10)
#counts_t.to_csv('topdeaths.csv')

MSOA Code,E02001347,E02001348,E02001349,E02001350,E02001351,E02001352,E02001353,E02001354,E02001355,E02001356,...,E02005123,E02005124,E02005125,E02005126,E02005127,E02005128,E02005129,E02005130,E02005131,Total
I25,0.00,5.00,6.00,6.00,5.00,5.00,5.00,5.00,5.00,5.00,...,6.00,5.00,5.00,7.00,0.00,5.00,5.00,5.00,5.00,"35,492.00"
C34,7.00,5.00,10.00,6.00,7.00,6.00,5.00,7.00,5.00,6.00,...,7.00,9.00,0.00,0.00,0.00,6.00,6.00,5.00,5.00,"29,690.00"
J44,5.00,9.00,7.00,13.00,5.00,5.00,0.00,7.00,7.00,5.00,...,12.00,5.00,0.00,0.00,0.00,12.00,5.00,8.00,5.00,"26,424.00"
J18,0.00,5.00,0.00,5.00,6.00,5.00,0.00,5.00,5.00,0.00,...,5.00,6.00,10.00,10.00,5.00,6.00,9.00,5.00,0.00,"24,569.00"
I21,0.00,5.00,6.00,5.00,0.00,0.00,5.00,5.00,0.00,5.00,...,5.00,0.00,5.00,5.00,0.00,5.00,5.00,0.00,0.00,"17,838.00"
I64,0.00,0.00,0.00,7.00,6.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,5.00,0.00,5.00,5.00,0.00,0.00,"10,780.00"
C50,0.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,5.00,0.00,0.00,5.00,0.00,5.00,0.00,"6,305.00"
C18,0.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,0.00,0.00,...,0.00,0.00,5.00,0.00,0.00,0.00,0.00,0.00,5.00,"4,482.00"
C15,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,"3,259.00"
I67,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,716.00"


In [7]:
#drop rows that aren't preventable
#causes = causes[causes.Treatable != 0]
#reset the indexes
#causes = causes.reset_index(drop=True)
counts_t = counts_t.reset_index()
#merge the dataframes - I merged left with the causes counts so those that are not treatable or preventable are dropped
counts_tp = causes.merge(counts_t, how='left', left_on ='ICD-10 codes', right_on = 'index')
counts_tp.sample(20, random_state=42)

,Condition Group,Cause,ICD-10 codes,Age,Treatable,Preventable,index,E02001347,E02001348,E02001349,...,E02005123,E02005124,E02005125,E02005126,E02005127,E02005128,E02005129,E02005130,E02005131,Total
365,"Pregnancy, childbirth and the perinatal period","Pregnancy, childbirth and the puerperium",O93,0-74,1.00,0.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
655,Injuries,Accidental Injuries,W30,0-74,0.00,1.00,W30,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
828,Injuries,Assault,X98,0-74,0.00,1.00,X98,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
692,Injuries,Accidental Injuries,W67,0-74,0.00,1.00,W67,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
213,Diseases of the respiratory system,Acute lower respiratory infections,J22,0-74,1.00,0.00,J22,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,429.00"
704,Injuries,Accidental Injuries,W79,0-74,0.00,1.00,W79,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
380,"Pregnancy, childbirth and the perinatal period",Certain conditions originating in the perinata...,P08,0-74,1.00,0.00,P08,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
266,Diseases of the genitourinary system,Inflammatory diseases of genitourinary system,N73,0-74,1.00,0.00,N73,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
280,"Pregnancy, childbirth and the perinatal period","Pregnancy, childbirth and the puerperium",O08,0-74,1.00,0.00,O08,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
39,Infectious diseases,HIV/AIDS,B20,0-74,0.00,1.00,B20,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


The index matches the ICD-10 codes in the above dataframe, however there are some NaN values. This section investigates the NaNs.

In [8]:
print(f"Data frame is {counts_tp.shape[0]:,} x {counts_tp.shape[1]}")

Data frame is 876 x 7156


In [9]:
counts_tp.isnull().sum()

Condition Group     0
Cause               0
ICD-10 codes        0
Age                 0
Treatable           0
                   ..
E02005128          89
E02005129          89
E02005130          89
E02005131          89
Total              89
Length: 7156, dtype: int64

In [10]:
unmatched = counts_tp[counts_tp['index'].isnull()]
NanICDs = unmatched['ICD-10 codes'].tolist()
NanICDs

['I13',
 'I03',
 'I04',
 'J92',
 'O09',
 'O17',
 'O18',
 'O19',
 'O27',
 'O37',
 'O38',
 'O39',
 'O49',
 'O50',
 'O51',
 'O52',
 'O53',
 'O54',
 'O55',
 'O56',
 'O57',
 'O58',
 'O59',
 'O76',
 'O77',
 'O78',
 'O79',
 'O93',
 'P06',
 'P09',
 'P16',
 'P17',
 'P18',
 'P19',
 'P30',
 'P31',
 'P32',
 'P33',
 'P34',
 'P40',
 'P41',
 'P42',
 'P43',
 'P44',
 'P45',
 'P46',
 'P47',
 'P48',
 'P49',
 'P62',
 'P63',
 'P64',
 'P65',
 'P66',
 'P67',
 'P68',
 'P69',
 'P73',
 'P79',
 'P82',
 'P84',
 'P85',
 'P86',
 'P87',
 'P88',
 'P89',
 'Q00 ',
 'Y67',
 'Y68',
 'Y83',
 'Y84',
 'V07',
 'V08',
 'W47',
 'W48',
 'W61',
 'W62',
 'W63',
 'W71',
 'W72',
 'W82',
 'W95',
 'W96',
 'W97',
 'W98',
 'X07',
 'X55',
 'X56',
 'Y09']

Because the preventable and treatable percentages were provided with ranges of ICD-10 codes, it is likely that when I amended
to 'long format' I added some codes that do not exist. I assumed each letter had 100 numbers associated. This is not true.

Therefore, NaNs will be dropped. 

In [11]:
counts_tp.dropna(subset = ["index"], inplace=True)

In [12]:
#sort dataframe by most prevalant preventable causes
#counts_tp['PreventableCount'] = counts_tp['Total']*counts_tp['Preventable']
#counts_tp.sort_values(by=['PreventableCount'],ascending=False, inplace=True)

#multiply all msoas by preventable %
counts_tp[msoas] = counts_tp[msoas].multiply(counts_tp['Preventable'], axis = 0)

#drop unneccessary columns
counts_top = counts_tp.drop(['Age','Treatable', 'Preventable', 'Total','index'], axis=1)
counts_top.loc[counts_top['Cause'] == 'Accidental Injuries']

,Condition Group,Cause,ICD-10 codes,E02001347,E02001348,E02001349,E02001350,E02001351,E02001352,E02001353,...,E02005122,E02005123,E02005124,E02005125,E02005126,E02005127,E02005128,E02005129,E02005130,E02005131
625,Injuries,Accidental Injuries,W00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
626,Injuries,Accidental Injuries,W01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
627,Injuries,Accidental Injuries,W02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
628,Injuries,Accidental Injuries,W03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
629,Injuries,Accidental Injuries,W04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,Injuries,Accidental Injuries,X53,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
772,Injuries,Accidental Injuries,X54,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
775,Injuries,Accidental Injuries,X57,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
776,Injuries,Accidental Injuries,X58,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [13]:
#create total column
counts_top['Total'] = counts_top[msoas].sum(axis=1)
counts_top = counts_top.fillna(0)
counts_top.sample(10, random_state=42)

,Condition Group,Cause,ICD-10 codes,E02001347,E02001348,E02001349,E02001350,E02001351,E02001352,E02001353,...,E02005123,E02005124,E02005125,E02005126,E02005127,E02005128,E02005129,E02005130,E02005131,Total
609,Injuries,Transport Accidents,V84,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
39,Infectious diseases,HIV/AIDS,B20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
215,Diseases of the respiratory system,Asthma and bronchiectasis,J46,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
203,Diseases of the respiratory system,Upper respiratory infections,J37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
239,Diseases of the digestive system,Cholelithiasis and cholecystitis,K80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
219,Diseases of the respiratory system,Abscess of lung and mediastinum pyothorax,J85,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
621,Injuries,Transport Accidents,V96,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
379,"Pregnancy, childbirth and the perinatal period",Certain conditions originating in the perinata...,P07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
312,"Pregnancy, childbirth and the perinatal period","Pregnancy, childbirth and the puerperium",O40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
138,Diseases of the circulatory system,Ischaemic heart diseases,I21,0.00,2.50,3.00,2.50,0.00,0.00,2.50,...,2.50,0.00,2.50,2.50,0.00,2.50,2.50,0.00,0.00,"8,919.00"


In [14]:
#create proportions (%) of deaths per preventable cause
#counts_top[msoas] = counts_top[msoas].divide(counts_top['Total'], axis=0)
#counts_top = counts_top.fillna(0)
#counts_top.sample(10, random_state=42)

### Secondary Analysis - checking causes with aggregated data 

In [15]:
#group data by condition group
counts_tp1 = counts_top.drop(['Cause','ICD-10 codes'], axis=1)
counts_agg_cg = counts_tp1.groupby('Condition Group').sum()
counts_agg_cg.sort_values(by=['Total'],ascending=False, inplace=True)
print(f"Data frame is {counts_agg_cg.shape[0]:,} x {counts_agg_cg.shape[1]}")

Data frame is 12 x 7149


In [16]:
counts_agg_cg.head(10)

,E02001347,E02001348,E02001349,E02001350,E02001351,E02001352,E02001353,E02001354,E02001355,E02001356,...,E02005123,E02005124,E02005125,E02005126,E02005127,E02005128,E02005129,E02005130,E02005131,Total
Condition Group,,,,,,,,,,,,,,,,,,,,,
Neoplasms,12.00,5.00,10.00,6.00,7.00,6.00,5.00,7.00,5.00,6.00,...,12.00,9.00,0.00,0.00,0.00,6.00,11.00,5.00,5.00,"36,746.50"
Diseases of the circulatory system,0.00,5.00,6.00,9.00,5.50,2.50,5.00,5.00,2.50,5.00,...,5.50,5.00,5.00,8.50,0.00,7.50,10.00,2.50,2.50,"35,313.50"
Diseases of the respiratory system,5.00,9.00,7.00,13.00,5.00,5.00,0.00,7.00,7.00,5.00,...,12.00,5.00,0.00,0.00,0.00,12.00,5.00,8.00,5.00,"26,734.00"
Injuries,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,634.00"
Alcohol-related and drug-related deaths,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,078.00"
Endocrine and metabolic diseases,0.00,0.00,0.00,2.50,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,2.50,2.50,0.00,0.00,594.00
Infectious diseases,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00
Adverse effects of medical and surgical care,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Congenital malformations,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [17]:
#group data by Cause group
counts_tp2 = counts_top.drop(['Condition Group','ICD-10 codes'], axis=1)
counts_agg_c = counts_tp2.groupby('Cause').sum()
counts_agg_c.sort_values(by=['Total'],ascending=False, inplace=True)
print(f"Data frame is {counts_agg_c.shape[0]:,} x {counts_agg_c.shape[1]}")

Data frame is 87 x 7149


In [18]:
counts_agg_c.head(10)

,E02001347,E02001348,E02001349,E02001350,E02001351,E02001352,E02001353,E02001354,E02001355,E02001356,...,E02005123,E02005124,E02005125,E02005126,E02005127,E02005128,E02005129,E02005130,E02005131,Total
Cause,,,,,,,,,,,,,,,,,,,,,
Lung cancer,7.00,5.00,10.00,6.00,7.00,6.00,5.00,7.00,5.00,6.00,...,7.00,9.00,0.00,0.00,0.00,6.00,6.00,5.00,5.00,"29,690.00"
Ischaemic heart diseases,0.00,5.00,6.00,5.50,2.50,2.50,5.00,5.00,2.50,5.00,...,5.50,2.50,5.00,6.00,0.00,5.00,5.00,2.50,2.50,"26,667.50"
Chronic lower respiratory diseases,5.00,9.00,7.00,13.00,5.00,5.00,0.00,7.00,7.00,5.00,...,12.00,5.00,0.00,0.00,0.00,12.00,5.00,8.00,5.00,"26,484.00"
Cerebrovascular diseases,0.00,0.00,0.00,3.50,3.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,2.50,0.00,2.50,5.00,0.00,0.00,"7,479.50"
Oesophageal cancer,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,"3,259.00"
Accidental Injuries,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,119.00"
Alcohol-specific disorders and poisonings,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,623.00"
Liver cancer,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,411.00"
Bladder cancer,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,349.00"


## Aggregated by cause group selected

The data has been aggregated two ways in the previous section. The aggregated by cause iteration was selected and is carried forward below. This final section takes the top 6 causes and sums all remaining to create 'other' category. 

In [19]:
remaining_causes = counts_agg_c.iloc[6:]
remaining_causes.loc['Other'] = remaining_causes.sum()
remaining_causes.head(10)

C:\Users\Yasmine Hujair\anaconda32020\envs\SpatialData\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,E02001347,E02001348,E02001349,E02001350,E02001351,E02001352,E02001353,E02001354,E02001355,E02001356,...,E02005123,E02005124,E02005125,E02005126,E02005127,E02005128,E02005129,E02005130,E02005131,Total
Cause,,,,,,,,,,,,,,,,,,,,,
Alcohol-specific disorders and poisonings,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,623.00"
Liver cancer,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,411.00"
Bladder cancer,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,349.00"
Aortic aneurysm,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,2.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,166.50"
Stomach cancer,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,625.00
Diabetes mellitus,0.00,0.00,0.00,2.50,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,2.50,2.50,0.00,0.00,594.00
Intentional self-harm,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,470.00
Drug disorders and poisonings,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,455.00
Lung diseases due to external agents,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,235.00


In [20]:
counts_agg_c_top = counts_agg_c.iloc[:6,:]
counts_agg_c_top.head(6)

,E02001347,E02001348,E02001349,E02001350,E02001351,E02001352,E02001353,E02001354,E02001355,E02001356,...,E02005123,E02005124,E02005125,E02005126,E02005127,E02005128,E02005129,E02005130,E02005131,Total
Cause,,,,,,,,,,,,,,,,,,,,,
Lung cancer,7.00,5.00,10.00,6.00,7.00,6.00,5.00,7.00,5.00,6.00,...,7.00,9.00,0.00,0.00,0.00,6.00,6.00,5.00,5.00,"29,690.00"
Ischaemic heart diseases,0.00,5.00,6.00,5.50,2.50,2.50,5.00,5.00,2.50,5.00,...,5.50,2.50,5.00,6.00,0.00,5.00,5.00,2.50,2.50,"26,667.50"
Chronic lower respiratory diseases,5.00,9.00,7.00,13.00,5.00,5.00,0.00,7.00,7.00,5.00,...,12.00,5.00,0.00,0.00,0.00,12.00,5.00,8.00,5.00,"26,484.00"
Cerebrovascular diseases,0.00,0.00,0.00,3.50,3.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,2.50,0.00,2.50,5.00,0.00,0.00,"7,479.50"
Oesophageal cancer,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,"3,259.00"
Accidental Injuries,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,119.00"


In [21]:
#append 'other' row
df2 = pd.DataFrame(remaining_causes, index=['Other'])
counts_agg_c_top = counts_agg_c_top.append(df2)
truecounts = counts_agg_c_top
truecounts = truecounts.T
truecounts = truecounts.reset_index()
truecounts = truecounts.rename(columns = {'index':'MSOACode'})
truecounts.head(10)

,MSOACode,Lung cancer,Ischaemic heart diseases,Chronic lower respiratory diseases,Cerebrovascular diseases,Oesophageal cancer,Accidental Injuries,Other
0,E02001347,7.00,0.00,5.00,0.00,5.00,0.00,0.00
1,E02001348,5.00,5.00,9.00,0.00,0.00,0.00,0.00
2,E02001349,10.00,6.00,7.00,0.00,0.00,0.00,0.00
3,E02001350,6.00,5.50,13.00,3.50,0.00,0.00,2.50
4,E02001351,7.00,2.50,5.00,3.00,0.00,0.00,0.00
5,E02001352,6.00,2.50,5.00,0.00,0.00,0.00,0.00
6,E02001353,5.00,5.00,0.00,0.00,0.00,0.00,0.00
7,E02001354,7.00,5.00,7.00,0.00,0.00,0.00,0.00
8,E02001355,5.00,2.50,7.00,0.00,0.00,0.00,0.00
9,E02001356,6.00,5.00,5.00,0.00,0.00,0.00,0.00


In [22]:
#transpose df
counts_agg_c_top = counts_agg_c_top.T

counts_agg_c_top['Total'] = counts_agg_c_top.sum(axis=1) #create total deaths per msoa row

counts_agg_c_top

,Lung cancer,Ischaemic heart diseases,Chronic lower respiratory diseases,Cerebrovascular diseases,Oesophageal cancer,Accidental Injuries,Other,Total
E02001347,7.00,0.00,5.00,0.00,5.00,0.00,0.00,17.00
E02001348,5.00,5.00,9.00,0.00,0.00,0.00,0.00,19.00
E02001349,10.00,6.00,7.00,0.00,0.00,0.00,0.00,23.00
E02001350,6.00,5.50,13.00,3.50,0.00,0.00,2.50,30.50
E02001351,7.00,2.50,5.00,3.00,0.00,0.00,0.00,17.50
...,...,...,...,...,...,...,...,...
E02005128,6.00,5.00,12.00,2.50,0.00,0.00,2.50,28.00
E02005129,6.00,5.00,5.00,5.00,5.00,0.00,2.50,28.50
E02005130,5.00,2.50,8.00,0.00,0.00,0.00,0.00,15.50
E02005131,5.00,2.50,5.00,0.00,0.00,0.00,0.00,12.50


In [23]:
columns = ['Lung cancer', 'Ischaemic heart diseases', 'Chronic lower respiratory diseases', 'Cerebrovascular diseases',
       'Oesophageal cancer', 'Accidental Injuries', 'Other']

#calculate proportions of deaths per category per msoa
counts_agg_c_top[columns] = counts_agg_c_top[columns].divide(counts_agg_c_top['Total'], axis=0)
counts_agg_c_top

,Lung cancer,Ischaemic heart diseases,Chronic lower respiratory diseases,Cerebrovascular diseases,Oesophageal cancer,Accidental Injuries,Other,Total
E02001347,0.41,0.00,0.29,0.00,0.29,0.00,0.00,17.00
E02001348,0.26,0.26,0.47,0.00,0.00,0.00,0.00,19.00
E02001349,0.43,0.26,0.30,0.00,0.00,0.00,0.00,23.00
E02001350,0.20,0.18,0.43,0.11,0.00,0.00,0.08,30.50
E02001351,0.40,0.14,0.29,0.17,0.00,0.00,0.00,17.50
...,...,...,...,...,...,...,...,...
E02005128,0.21,0.18,0.43,0.09,0.00,0.00,0.09,28.00
E02005129,0.21,0.18,0.18,0.18,0.18,0.00,0.09,28.50
E02005130,0.32,0.16,0.52,0.00,0.00,0.00,0.00,15.50
E02005131,0.40,0.20,0.40,0.00,0.00,0.00,0.00,12.50


In [24]:
#convert nans to 0s (if there are no deaths there are no deaths!)
counts_agg_c_top = counts_agg_c_top.fillna(0)
counts_agg_c_top

,Lung cancer,Ischaemic heart diseases,Chronic lower respiratory diseases,Cerebrovascular diseases,Oesophageal cancer,Accidental Injuries,Other,Total
E02001347,0.41,0.00,0.29,0.00,0.29,0.00,0.00,17.00
E02001348,0.26,0.26,0.47,0.00,0.00,0.00,0.00,19.00
E02001349,0.43,0.26,0.30,0.00,0.00,0.00,0.00,23.00
E02001350,0.20,0.18,0.43,0.11,0.00,0.00,0.08,30.50
E02001351,0.40,0.14,0.29,0.17,0.00,0.00,0.00,17.50
...,...,...,...,...,...,...,...,...
E02005128,0.21,0.18,0.43,0.09,0.00,0.00,0.09,28.00
E02005129,0.21,0.18,0.18,0.18,0.18,0.00,0.09,28.50
E02005130,0.32,0.16,0.52,0.00,0.00,0.00,0.00,15.50
E02005131,0.40,0.20,0.40,0.00,0.00,0.00,0.00,12.50


In [25]:
#drop total & preventablecount rows
counts_agg_c_top = counts_agg_c_top.drop(['Total'], axis = 0)
#drop total column
counts_agg_c_top = counts_agg_c_top.drop(['Total'], axis = 1)
#reset index 
counts_agg_c_top = counts_agg_c_top.reset_index()
counts_agg_c_top = counts_agg_c_top.rename(columns = {'index':'MSOACode'})
counts_agg_c_top

,MSOACode,Lung cancer,Ischaemic heart diseases,Chronic lower respiratory diseases,Cerebrovascular diseases,Oesophageal cancer,Accidental Injuries,Other
0,E02001347,0.41,0.00,0.29,0.00,0.29,0.00,0.00
1,E02001348,0.26,0.26,0.47,0.00,0.00,0.00,0.00
2,E02001349,0.43,0.26,0.30,0.00,0.00,0.00,0.00
3,E02001350,0.20,0.18,0.43,0.11,0.00,0.00,0.08
4,E02001351,0.40,0.14,0.29,0.17,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...
7143,E02005127,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7144,E02005128,0.21,0.18,0.43,0.09,0.00,0.00,0.09
7145,E02005129,0.21,0.18,0.18,0.18,0.18,0.00,0.09
7146,E02005130,0.32,0.16,0.52,0.00,0.00,0.00,0.00


In [26]:
#export csv
counts_agg_c_top.to_csv('causes-of-preventable-deaths.csv')

## Create geojsons

In [27]:
#loading the MSOAs for England and Wales

msoas = gpd.read_file('https://github.com/jreades/i2p/blob/master/data/src/Middle_Layer_Super_Output_Areas__December_2011__EW_BGC_V2-shp.zip?raw=true')

#extract england from msoas
msoas_england = msoas[msoas.MSOA11CD.str.startswith('E')]

#drop unneccesary columns
to_drop = ['MSOA11NMW','LONG','LAT','Shape__Are','Shape__Len']
msoas_england.drop(columns=to_drop, inplace = True)

#tidy up boroughs
msoas_england['borough'] = msoas_england.MSOA11NM.str.replace(r' \d+','',regex=True)

C:\Users\Yasmine Hujair\anaconda32020\envs\SpatialData\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Yasmine Hujair\anaconda32020\envs\SpatialData\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [28]:
#pull in msoa nicknames
msoa_nms = pd.read_csv('https://github.com/jreades/i2p/blob/master/data/src/MSOA-Names-1.8.csv.gz?raw=true', compression='gzip')

#merge dataframes
msoas = pd.merge(msoas_england, msoa_nms, how = 'inner' , left_on = 'MSOA11CD', right_on = 'msoa11cd')
# tidy up
to_drop = ['msoa11cd','msoa11nm','msoa11nmw','Laname','msoa11hclnmw']
msoas.drop(columns=to_drop, inplace=True)
print(msoas.shape)

(6791, 8)


In [29]:
#create geodataframe
gdf = pd.merge(msoas, counts_agg_c_top, left_on = 'MSOA11CD', right_on = 'MSOACode', how = 'inner')

In [30]:
#export to geojson

gdf.to_file(os.path.join('Data','preventabledeathsbycause.geojson'), driver='GeoJSON')

The team also requested true counts as well as proportions in geojson and csv format - created below.

In [34]:
#creating geojson of proportions and counts together
props_and_counts = pd.merge(truecounts, counts_agg_c_top, on='MSOACode', how = 'left', suffixes = ('_count', '_prop'))
props_and_counts.head()

,MSOACode,Lung cancer_count,Ischaemic heart diseases_count,Chronic lower respiratory diseases_count,Cerebrovascular diseases_count,Oesophageal cancer_count,Accidental Injuries_count,Other_count,Lung cancer_prop,Ischaemic heart diseases_prop,Chronic lower respiratory diseases_prop,Cerebrovascular diseases_prop,Oesophageal cancer_prop,Accidental Injuries_prop,Other_prop
0,E02001347,7.00,0.00,5.00,0.00,5.00,0.00,0.00,0.41,0.00,0.29,0.00,0.29,0.00,0.00
1,E02001348,5.00,5.00,9.00,0.00,0.00,0.00,0.00,0.26,0.26,0.47,0.00,0.00,0.00,0.00
2,E02001349,10.00,6.00,7.00,0.00,0.00,0.00,0.00,0.43,0.26,0.30,0.00,0.00,0.00,0.00
3,E02001350,6.00,5.50,13.00,3.50,0.00,0.00,2.50,0.20,0.18,0.43,0.11,0.00,0.00,0.08
4,E02001351,7.00,2.50,5.00,3.00,0.00,0.00,0.00,0.40,0.14,0.29,0.17,0.00,0.00,0.00


In [35]:
props_and_counts.to_csv('causes-proportions-and-counts-of-preventable-deaths.csv')

In [36]:
#create geodataframe
gdf = pd.merge(msoas, props_and_counts, left_on = 'MSOA11CD', right_on = 'MSOACode', how = 'inner')

#export to geojson
gdf.to_file(os.path.join('Data','causes-proportions-and-counts-of-preventable-deaths.geojson'), driver='GeoJSON')

Any questions speak to [Yasmine Hujair](yasmine.hujair@gmail.com)